# Import Packages

In [56]:
from sentence_transformers import SentenceTransformer
from tqdm import tqdm
import requests 
import numpy as np
import pandas as pd

# Q1

In [1]:
model_name = "multi-qa-distilbert-cos-v1"

In [3]:
embedding_model = SentenceTransformer(model_name)

In [4]:
user_question = "I just discovered the course. Can I still join it?"

In [5]:
vec = embedding_model.encode([user_question])

In [7]:
vec.shape

(1, 768)

In [36]:
np.dot(vec,vec.T)

array([[1.0000001]], dtype=float32)

In [12]:
vec[:,0]

array([0.07822265], dtype=float32)

# Q2

In [14]:
base_url = 'https://github.com/DataTalksClub/llm-zoomcamp/blob/main'
relative_url = '03-vector-search/eval/documents-with-ids.json'
docs_url = f'{base_url}/{relative_url}?raw=1'
docs_response = requests.get(docs_url)
documents = docs_response.json()

In [16]:
len(documents)

948

In [18]:
target = "machine-learning-zoomcamp"

selected_docs = [ doc for doc in documents if doc["course"]==target]

In [21]:
selected_docs[0]

{'text': 'Machine Learning Zoomcamp FAQ\nThe purpose of this document is to capture frequently asked technical questions.\nWe did this for our data engineering course and it worked quite well. Check this document for inspiration on how to structure your questions and answers:\nData Engineering Zoomcamp FAQ\nIn the course GitHub repository there’s a link. Here it is: https://airtable.com/shryxwLd0COOEaqXo\nwork',
 'section': 'General course-related questions',
 'question': 'How do I sign up?',
 'course': 'machine-learning-zoomcamp',
 'id': '0227b872'}

In [25]:

qa_text = [f"{doc['question']} {doc['text']}" for doc in selected_docs]
embeddings = embedding_model.encode(qa_text)

In [26]:
embeddings.shape

(375, 768)

# Q3

In [29]:
embeddings.dot(embeddings.T).shape

(375, 375)

In [32]:
np.trace(np.dot(embeddings,embeddings.T))

375.0

In [33]:
scores = np.dot(embeddings,vec.T)

In [34]:
scores.shape

(375, 1)

In [38]:
scores.max(),scores.min()

(0.65065765, -0.14676371)

# Q4

In [45]:
class VectorSearchEngine():
    def __init__(self, documents, embeddings):
        self.documents = documents
        self.embeddings = embeddings

    def search(self, v_query, num_results=10):
        scores = self.embeddings.dot(v_query)
        idx = np.argsort(-scores[:,0])[:num_results]
        return [self.documents[i] for i in idx]

search_engine = VectorSearchEngine(documents=selected_docs, embeddings=embeddings)
retrieved_docs = search_engine.search(vec.T , num_results=5)

In [47]:


base_url = 'https://github.com/DataTalksClub/llm-zoomcamp/blob/main'
relative_url = '03-vector-search/eval/ground-truth-data.csv'
ground_truth_url = f'{base_url}/{relative_url}?raw=1'

df_ground_truth = pd.read_csv(ground_truth_url)
df_ground_truth = df_ground_truth[df_ground_truth.course == 'machine-learning-zoomcamp']
ground_truth = df_ground_truth.to_dict(orient='records')

In [59]:
relevance_total = []

for q in tqdm(ground_truth):
    doc_id = q['document']
    
    vec = embedding_model.encode([q['question']])
    results = search_engine.search(vec.T , num_results=5)
    
    relevance = [d['id'] == doc_id for d in results]
    relevance_total.append(relevance)

100%|██████████| 1830/1830 [03:15<00:00,  9.35it/s]


In [60]:
def hit_rate(relevance_total):
    cnt = 0

    for line in relevance_total:
        if True in line:
            cnt = cnt + 1

    return cnt / len(relevance_total)

In [61]:
hit_rate(relevance_total)

0.9398907103825137

# Q5

In [ ]:
from elasticsearch import Elasticsearch

es_client = Elasticsearch('http://localhost:9200') 

index_settings = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "text": {"type": "text"},
            "section": {"type": "text"},
            "question": {"type": "text"},
            "course": {"type": "keyword"},
            "id": {"type": "keyword"},
        }
    }
}